#### Implementation of Huffman Coding

In [11]:
import heapq
import os

class BinaryTreeNode:
    
    def __init__(self,value,freq):
        self.value = value
        self.freq = freq
        self.left = None
        self.right = None
        
    #to eliminate confusion in comparing btw 2 BT nodes
    def __lt__(self,other):
        
        return self.freq < other.freq
    
    def __equal__(self,other):
        
        return self.freq == other.freq
        
class HuffmanCoding:
    
    def __init__(self,path):
        self.path = path
        self.__heap = []
        self.__codes = {}
        self.__reversecodes = {}
        
    def __makeFreqDict(self,text):
        
        freq_dict = {}
        for char in text:
            
            if char not in freq_dict:
                freq_dict[char] = 0
                
            freq_dict[char] += 1
        
        return freq_dict
    
    def __buildHeap(self,freq_dict):
        #maintaining a heap of Binary Tree nodes (every ele is a BT node)
        
        #creating a BT Node w key and freq and pushing the node in the heap
        for key in freq_dict:
            freq = freq_dict[key]
            btnode = BinaryTreeNode(key,freq)
            heapq.heappush(self.__heap,btnode)
            
    def __buildTree(self):
        
        #loop runs till only one node is left in the heap
        #that last node is the node with freq = sum of highest freq node and sum of 2nd and 3rd highest freq nodes
        while (len(self.__heap) > 1):
            btnode1 = heapq.heappop(self.__heap)
            btnode2 = heapq.heappop(self.__heap)
            
            freqsum = btnode1.freq + btnode2.freq
            newnode = BinaryTreeNode(None, freqsum)
            newnode.left = btnode1
            newnode.right = btnode2
            heapq.heappush(self.__heap, newnode)
            
        return
    
    def __buildCodesHelper(self,root,currbits):
        
        if root is None:
            return
        
        #for all char nodes
        if root.value is not None:
            self.__codes[root.value] = currbits
            self.__reversecodes[currbits] = root.value
            return
        
        #for all nodes that just contain freq sum
        self.__buildCodesHelper(root.left,currbits+'0')
        self.__buildCodesHelper(root.right,currbits+'1')
        
    def __buildCodes(self):
        
        root = heapq.heappop(self.__heap)
        
        self.__buildCodesHelper(root,'')

    def __getEncodedText(self,text):
        
        encodedtext = ''
        for char in text:
            encodedtext += self.__codes[char]
            
        return encodedtext
    
    def __paddedEncodedText(self,encodedtext):
        
        #number of zeroes to be padded
        padded0 = 8 - (len(encodedtext)%8)
        
        for i in range(padded0):
            encodedtext += '0'
            
        #store padded 0 number in the front
        paddedinfo = "{0:08b}".format(padded0)
        #0:08b meaning - 1. 0 is the index of arg in the format function (can give multiple)
        #2. 08b - convert the arg into 8 bits code
        
        paddedencodedtext = paddedinfo + encodedtext
        
        return paddedencodedtext
    
    def __bytesarr(self,encodedtext):
        
        arr = []
        
        for i in range(0,len(encodedtext),8):
            byte = encodedtext[i:i+8]
            #foll converts above bit string that is stored in byte in binary (eg: 10110101) 
            #and ',2' converts it to integer w base 2 (i.e 102 etc etc)
            arr.append(int(byte,2))
            
        return arr
    
    def compress(self):
        #get file from path
        
        #read text from file
        filename, fileextension = os.path.splitext(self.path)
        outputpath = filename + '.bin'
        #text = 'fasbsajfhbsafjhbashf'
        
        with open(self.path, '+r') as file, open(outputpath, 'wb') as output:
        
            text = file.read()
            #to remove trailing spaces
            text = text.rstrip()

            #make freq dict
            freq_dict = self.__makeFreqDict(text)

            #construct the heap from the freq dict
            self.__buildHeap(freq_dict)

            #construct the binary tree from the heap
            self.__buildTree()

            #generate codes from the tree
            self.__buildCodes()

            #create encoded text
            encodedtext = self.__getEncodedText(text)

            #pad the encoded text
            paddedencoded = self.__paddedEncodedText(encodedtext)

            #get the bytes array
            bytesarr = self.__bytesarr(paddedencoded)

            #put encoded text into the binary file
            finalbytes = bytes(bytesarr)

            #return the binary file as output
            output.write(finalbytes)
            print('compressed')
        return outputpath
    
    
    def __removePadding(self,bit_str):
        
        paddedinfo = bit_str[:8]
        extrapaddedinfo = int(paddedinfo,2) #convert bin to int to get number of 0s at the end
        
        bit_str = bit_str[8:]
        textwopadding = bit_str[:-1*extrapaddedinfo] #remove padding from the end
        
        return textwopadding
    
    def __decodetext(self,actualtext):
        
        decodedtext = ''
        currbit = ''
        
        for bit in actualtext:
            currbit += bit
            if currbit in self.__reversecodes:
                char =  self.__reversecodes[currbit]
                decodedtext += char
                currbit = ''
                
        return decodedtext
        
    def decompress(self,inputPath):
        
        filename, fileext = os.path.splitext(inputPath)
        outputPath = filename + '_decompressed' + '.txt'
        with open(inputPath, 'rb') as file, open(outputPath, 'w') as output:
            bit_str = ''
            byte = file.read(1) #read 1 byte at a time
            while byte: #byte is in the form - '0x1f' etc
                byte = ord(byte) #converted into int like 102 etc
                bits = bin(byte)[2:].rjust(8,'0') #bin converts 102 to b/101 (bit form)..
                #..[2:] done to start reading from 101 in 'b/101' to add 0s and make 8 bits using rjust method
                bit_str += bits
                byte = file.read(1)
                
            #remove padding from the bit string
            actualtext = self.__removePadding(bit_str)
            
            #decode the actual text
            decodedtext = self.__decodetext(actualtext)
            
            output.write(decodedtext)
            
        return
                
        
        
path = 'C:/Users/memoh/OneDrive/Desktop/HuffmanTest.txt' 
h = HuffmanCoding(path)
outputPath = h.compress()
h.decompress(outputPath)


compressed
